# Flight Router Sandbox

Interactive notebook for exploring the Flight Router framework.

**This notebook uses a curated demo dataset** (`data/demo_flights.db`) containing ~89k real airline flights (easyJet, British Airways, LOT, etc.). Test carriers have been excluded for realistic routing.

**Key concepts:**
- Multi-criteria Dijkstra algorithm finds Pareto-optimal routes (no route dominates another in both cost AND time)
- Routes are round-trips: origin → destinations → origin
- Search performance: ~3-5 seconds for the demo dataset

## 1. Setup & Initialization

In [ ]:
import sys
from datetime import datetime, timedelta
from pathlib import Path

# Setup path to import flight_router from monorepo
# This notebook lives in: src/flight_router/examples/
# Project root is 3 levels up
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent.parent

# Add project root to Python path if not already there
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.flight_router.application import FindOptimalRoutes

# Path to the curated demo database (relative to notebook location)
DEMO_DB = Path("data/demo_flights.db")

if not DEMO_DB.exists():
    raise FileNotFoundError(
        f"Demo database not found at {DEMO_DB}. "
        "Make sure you're running this notebook from the examples/ directory."
    )

# Initialize the router with the demo database
router = FindOptimalRoutes(db_path=DEMO_DB)

print("=" * 50)
print("FLIGHT ROUTER SANDBOX")
print("=" * 50)
print(f"\nDatabase: {DEMO_DB}")
print(f"Algorithm: {router.algorithm_name}")
print(f"Total airports: {len(router.get_available_airports())}")
print(f"Ready: {router.is_ready}")

## 2. Discover Available Dates (CRITICAL!)

Your searches must use dates within the database range, otherwise you'll get 0 results.

In [ ]:
import sqlite3
import pandas as pd

# Check what dates are in the demo database
conn = sqlite3.connect(DEMO_DB)
df = pd.read_sql('''
    SELECT 
        MIN(departure_date) as earliest_flight,
        MAX(departure_date) as latest_flight,
        COUNT(*) as total_quotes
    FROM flight_quotes
''', conn)

# Check carriers
carriers = pd.read_sql('''
    SELECT carrier_name, COUNT(*) as routes
    FROM flights_static 
    GROUP BY carrier_name 
    ORDER BY routes DESC 
    LIMIT 5
''', conn)
conn.close()

print("=" * 50)
print("DEMO DATABASE INFO")
print("=" * 50)
print(f"Total flight quotes: {df['total_quotes'].iloc[0]:,}")
print(f"Earliest departure:  {df['earliest_flight'].iloc[0]}")
print(f"Latest departure:    {df['latest_flight'].iloc[0]}")
print()
print("Top 5 carriers:")
for _, row in carriers.iterrows():
    print(f"  {row['carrier_name']}: {row['routes']} routes")
print()
print("⚠️  Your searches must use dates WITHIN this range!")

## 3. Discover Available Airports

In [ ]:
airports = sorted(router.get_available_airports())
print(f"Available airports ({len(airports)} total):\n")

# Check major hubs
print("Major hubs availability:")
major = ['WAW', 'LHR', 'CDG', 'FRA', 'AMS', 'MUC', 'FCO', 'VIE', 'ZRH', 'BRU', 'BCN', 'MAD']
for code in major:
    status = "✓" if code in airports else "✗"
    print(f"  {status} {code}")

print("\nAll airports:")
for i in range(0, len(airports), 15):
    print(f"  {airports[i:i+15]}")

## 4. Check Direct Routes from Warsaw

In [ ]:
# Get cities with direct flights from Warsaw
reachable = sorted([
    city for city in router.get_available_airports() 
    if router.has_route("WAW", city)
])

print(f"Direct flights from WAW: {len(reachable)} destinations\n")
for i in range(0, len(reachable), 15):
    print(f"  {reachable[i:i+15]}")

## 5. Simple Search (Single Destination)

Find round-trip routes: WAW → LHR → WAW

In [ ]:
# Use dates that exist in the demo database (July 13-19, 2026)
results = router.search(
    origin="WAW",
    destinations={"LHR"},
    departure_date=datetime(2026, 7, 13),
    return_date=datetime(2026, 7, 19),
)

print(f"WAW → LHR → WAW: Found {len(results)} Pareto-optimal routes\n")

for route in results[:5]:
    print(f"Route: {' → '.join(route.route_cities)}")
    print(f"  Cost: €{route.total_cost:.2f}")
    print(f"  Duration: {route.total_time / 60:.1f} hours")
    print(f"  Segments: {route.num_segments}")
    print()

## 6. Inspect Route Details

In [ ]:
if results:
    route = results[0]  # Best route (lowest cost)
    print(f"Detailed breakdown for cheapest route:\n")
    
    for seg in route.segments:
        dep_dt = router.epoch_minutes_to_datetime(seg.dep_time)
        arr_dt = router.epoch_minutes_to_datetime(seg.arr_time)
        
        print(f"  ✈ {seg.departure_airport} → {seg.arrival_airport}")
        print(f"    Depart:  {dep_dt.strftime('%a %Y-%m-%d %H:%M')}")
        print(f"    Arrive:  {arr_dt.strftime('%a %Y-%m-%d %H:%M')}")
        print(f"    Duration: {seg.duration / 60:.1f}h | Price: €{seg.price:.2f}")
        if seg.carrier_name:
            print(f"    Carrier: {seg.carrier_name}")
        print()
else:
    print("No routes found. Check your dates match the database range (Cell 2).")

## 7. Multi-City Trip

Visit multiple destinations before returning home.

In [ ]:
# Find available hub airports
hubs = {'LHR', 'CDG', 'FRA', 'AMS', 'MUC', 'FCO', 'VIE', 'ZRH', 'BRU', 'CPH'}
available_hubs = hubs & router.get_available_airports()
print(f"Available hub airports: {sorted(available_hubs)}\n")

# Pick two destinations
if len(available_hubs) >= 2:
    destinations = set(list(sorted(available_hubs))[:2])
    print(f"Searching: WAW → {destinations} → WAW")
    print("This may take 10-30 seconds for multi-city...\n")
    
    results = router.search(
        origin="WAW",
        destinations=destinations,
        departure_date=datetime(2026, 7, 13),
        return_date=datetime(2026, 7, 19),
    )
    
    print(f"Found {len(results)} Pareto-optimal routes\n")
    for route in results[:5]:
        print(f"  {' → '.join(route.route_cities)}")
        print(f"    Cost: €{route.total_cost:.2f} | Duration: {route.total_time / 60:.1f}h")
        print()
else:
    print("Not enough hub airports in database for multi-city search.")

## 8. Performance Benchmark

In [ ]:
import time

# Test search performance from different origins
available = router.get_available_airports()
origins = [o for o in ["WAW", "FRA", "LHR", "CDG"] if o in available]
dest = "VIE" if "VIE" in available else list(available)[0]

print(f"Search Performance (destination: {dest}):\n")
for origin in origins:
    if origin == dest:
        continue
    start = time.time()
    results = router.search(
        origin=origin,
        destinations={dest},
        departure_date=datetime(2026, 7, 13),
    )
    elapsed = time.time() - start
    print(f"  {origin} → {dest}: {len(results)} routes in {elapsed:.1f}s")

## 9. Raw API (Advanced)

Use epoch minutes directly for precise time window control.

In [ ]:
# Convert datetime to epoch minutes
t_min = router.datetime_to_epoch_minutes(datetime(2026, 7, 14, 8, 0))   # July 14, 8 AM
t_max = router.datetime_to_epoch_minutes(datetime(2026, 7, 16, 22, 0))  # July 16, 10 PM

print(f"Time window: {t_min:.0f} to {t_max:.0f} minutes since epoch")
print(f"That's {(t_max - t_min) / 60:.0f} hours\n")

# Pick a destination that exists
dest = "VIE" if "VIE" in router.get_available_airports() else "LHR"

results = router.search_raw(
    start_city="WAW",
    required_cities={dest},
    t_min=t_min,
    t_max=t_max,
)

print(f"Found {len(results)} routes in narrow {(t_max - t_min) / 60:.0f}h window")

## 10. Cleanup

Always shutdown to release database connections and background threads.

In [ ]:
router.shutdown()
print("✓ Router shutdown complete")

## Alternative: Context Manager Pattern

Auto-cleanup when the `with` block exits.

In [ ]:
# Preferred pattern for production use
with FindOptimalRoutes(db_path=DEMO_DB) as router:
    results = router.search(
        origin="WAW",
        destinations={"LHR"},
        departure_date=datetime(2026, 7, 13),
    )
    print(f"Found {len(results)} routes")

# Router automatically cleaned up here
print("✓ Auto-cleanup complete")

---

## Troubleshooting

| Issue | Cause | Solution |
|-------|-------|----------|
| `Airport 'XXX' not found` | Airport not in demo dataset | Check Cell 3 for available airports |
| `Found 0 routes` | Wrong date range | Demo data is July 13-19, 2026 |
| `FileNotFoundError` | Wrong working directory | Run notebook from `examples/` folder |
| Search takes 3-5s | Normal for 89k flights | Working on it :) |

---

## About the Demo Dataset

The `data/demo_flights.db` contains:
- **89,241 flight quotes** from real airlines
- **7,523 unique routes** across Europe
- **39 carriers** (easyJet, British Airways, LOT, Lufthansa, etc.)
- **Date range**: July 13-19, 2026

Test carriers (like "Duffel Airways") have been excluded for realistic routing demonstrations.